In [1]:
import time
from functools import reduce

import pandas as pd

In [2]:
# See lib.rs for details about constants
CARD_TYPES = ("DC", "CC")
TRANSACTION_TYPES = (
    "food-and-household",
    "home",
    "uncategorized",
    "leisure-and-lifestyle",
    "health-and-beauty",
    "shopping-and-services",
    "children",
    "vacation-and-travel",
    "education",
    "insurance",
    "investments-and-savings",
    "expenses-and-other",
    "cars-and-transportation",
)
CHANNELS = ("mobile", "web")


# Required time windows
WINDOWS_IN_DAYS = (
    7,  # week
    14,  # two weeks
    21,  # three weeks
    30,  # month
    90,  # three months
    180,  # half of the year
    360,  # two years
)


def generate_pivoted_batch(data: pd.DataFrame, t_minus: int, groups=list[str]) -> pd.DataFrame:
    pre_agg = (
        data
        .loc[data["t_minus"] <= t_minus]
        .groupby(["customer_id"] + groups, as_index=False)
        ["trx_amnt"].agg(["count", "mean", "sum", "min", "max"])
    )
    pivoted = pre_agg.pivot(
        columns=groups,
        index="customer_id",
        values=["count", "mean", "sum", "min", "max"],
    )
    pivoted.columns = ["_".join(a[1:]) + f"_{t_minus}d_{a[0]}" for a in pivoted.columns.to_flat_index()]
    return pivoted

In [3]:
# Rm an output parquet folder if it exists
!rm -r "../tmp_out"

rm: cannot remove '../tmp_out': No such file or directory


In [ ]:
path = "../test_data_small"
start_time = time.time()

data = pd.read_parquet(path)

#  #   Column       Dtype   
# ---  ------       -----   
#  0   customer_id  int64   
#  1   card_type    object  
#  2   trx_type     object  
#  3   channel      object  
#  4   trx_amnt     float64 
#  5   t_minus      int64   
#  6   part_col     category


dfs_list = []

for win in WINDOWS_IN_DAYS:
    # Iterate over combination card_type + trx_type
    dfs_list.append(generate_pivoted_batch(data, win, ["card_type", "trx_type"]))

    # Iterate over combination channel + trx_type
    dfs_list.append(generate_pivoted_batch(data, win, ["channel", "trx_type"]))

(
    reduce(lambda a, b: pd.merge(a, b, left_index=True, right_index=True), dfs_list)
    .reset_index(drop=False)
    .to_parquet("../tmp_out")
)

end_time = time.time()

In [ ]:
print(f"Total time: {end_time - start_time} seconds")

In [ ]:
pd.read_parquet("../tmp_out")